In [38]:
import pandas as pd
import numpy as np
import math
import numpy as np
from scipy.stats import norm
%matplotlib inline



In [39]:
mydata = pd.read_excel("Dataset.xlsx")
mydata.head()

,id,PurchasePrice,CurrentCoupon,CouponsRemaining,Time2Maturity,BondsHeld,YearlyRate,Principal,PD
0,1,92.489,7,73,29.310559,26000,1.00,157,0.021476
1,2,101.676,2,13,2.272379,5000,0.98,101,0.183483
2,3,107.328,6,76,5.663777,100000,2.81,105,0.006481
3,4,127.260,6,74,25.764823,21000,0.87,179,0.672002
4,5,95.201,5,60,9.176237,5000001,0.77,97,0.351326


In [146]:
#Q1 part a

# market interest rate

rate1_to_3 = 0.0146
rate3_to_5 = 0.0132
rate5_to_10 = 0.0128
rate_over_10 = 0.0144

# parameters
n = np.shape(mydata)[0]
current_price = np.zeros(np.shape(mydata)[0])

for i in range(n):
    X = mydata.Principal[i]/100
    num = math.floor((mydata.CouponsRemaining[i])/mydata.Time2Maturity[i])
    C = X * (mydata.YearlyRate[i]/100) / num
    r = 0
    if mydata.Time2Maturity[i] < 3:
        r = rate1_to_3
    elif (mydata.Time2Maturity[i] >= 3 and mydata.Time2Maturity[i]) < 5:
        r = rate3_to_5
    elif (mydata.Time2Maturity[i] >= 5 and mydata.Time2Maturity[i]) < 10:
        r = rate5_to_10
    else:
        r = rate_over_10
    # payment sequence
    payment_series = np.repeat(C, mydata.CouponsRemaining[i] + 1)
    payment_series[0] -= C
    payment_series[mydata.CouponsRemaining[i]] += X
    
    # PV calculation
    current_price[i] = np.npv(r, payment_series) #* mydata.BondsHeld[i]

    


C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [147]:
mydata['PresentPrice'] = current_price
current_price

array([0.90604186, 0.85982893, 0.50925212, ..., 0.58770667, 0.85544744,
       0.53823144])

In [7]:
xls = pd.ExcelFile('Dataset.xlsx')
bond = pd.read_excel(xls, 'Bond')
mortgage = pd.read_excel(xls, 'Mortgage')
loan = pd.read_excel(xls, 'RetailLoan')

loan.head()

,ID,OriginalAmount,Term,YearlyInterestRate,CurrentInstallment,PD
0,1,8305,23,0.19,6,0.862347
1,2,3380,22,0.26,10,0.003640
2,3,6955,35,0.29,17,0.122260
3,4,13443,23,0.10,14,0.839879
4,5,4037,22,0.15,9,0.010284


In [165]:
# part b

#total captital requirement
# for retail
def capital_requirement_retail(PD):
    # floor
    if (PD < 0.0003):
        PD = 0.0003
    R =  0.03 * ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) 
    R += 0.16 * (1 - ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= 1
    return(K) 


def capital_requirement_mortgage(PD, LGD):
    # floor
    if (PD < 0.0003):
        PD = 0.0003
    if (LGD < 0.1):
        LGD = 0.1
    R = 0.15
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    return K




In [149]:
len_bond = len(bond)
lgd_bond = np.zeros(len_bond)
for i in range(len_bond):
    if (bond.PD[i] < 3.6): 
        lgd_bond[i] = 0.45
    else:
        lgd_bond[i] = 0.75

K = 0
#cap = np.zeros(len_bond)
for i in range(len_bond):
    X = mydata.Principal[i]/100
    num = math.floor((mydata.CouponsRemaining[i])/mydata.Time2Maturity[i])
    C = X * (mydata.YearlyRate[i]/100) / num
    M = 0
    for j in range(bond.CouponsRemaining[i] - 1):
        if num != 0:
            M += C*(12/num)*(j+1)
        else:
            M = 0
    M = (M + (C + bond.Principal[i]))/(bond.CouponsRemaining[i]*C + bond.Principal[i])
    R = 0.12*((1-np.exp(-50*bond.PD[i]))/(1-np.exp(-50))) + 0.24*(1 - (1-np.exp(-50*bond.PD[i]))/(1-np.exp(-50)))
    b = (0.11852 - 0.05478 * np.log(bond.PD[i]))**2
    l = (norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(bond.PD[i]) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - bond.PD[i]) * ((1+(M +2.5)*b)/(1-1.5*b))*lgd_bond[i]
    if not(math.isnan(l)):
        #l = np.median(cap)
        K += l
    #cap[i] = l

    
    

C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


In [150]:

print(f"total capital requirement for bond is:{K}")

total capital requirement for bond is:24990.94424242525


In [151]:

len_mortgage = len(mortgage)
cap_mortgage = 0
for i in range(len_mortgage):
    cap_mortgage += capital_requirement_mortgage(mortgage.PD[i], mortgage.LGD[i])


    
len_loan = len(loan)
cap_loan = 0
for i in range(len_loan):
    cap_loan += capital_requirement_retail(loan.PD[i])
    #pro_loan += loan.PD[i] * 1 * (loan.OriginAmount[i] - 12*Origin.Term)
   


print(f"total capital requirement for mortgage is:{cap_mortgage}")
print(f"total capital requirement for loan is:{cap_loan}")



total capital requirement for mortgage is:3704.244407895515
total capital requirement for loan is:2725.8263040521633


In [152]:
# monthly payment for motgage and loan

monthly_mor = np.zeros(len_mortgage)

for i in range(len_mortgage):
    monthly_mor[i] = np.pmt(mortgage.Rate[i]/12, 12*mortgage.TermYears[i], mortgage.OriginalAmount[i])
    
# calculate the unpaid principal(EAD) for mortgage
EAD_mort = np.zeros(len_mortgage)

for i in range(len_mortgage):
    unpaid_principal = mortgage.OriginalAmount[i]
    for j in range(mortgage.CurrentInstallment[i]):
        unpaid_principal = unpaid_principal - monthly_mor[i] + unpaid_principal*mortgage.Rate[i]/12
    EAD_mort[i] = unpaid_principal - mortgage.Collateral_valuation[i]
    
    
monthly_lo = np.zeros(len_loan)

for i in range(len_loan):
    monthly_lo[i] = np.pmt(loan.YearlyInterestRate[i]/12, 12*loan.Term[i], loan.OriginalAmount[i])
    
# calculate the unpaid principal(EAD) for loan
EAD_loan = np.zeros(len_loan)

for i in range(len_loan):
    unpaid_principal = loan.OriginalAmount[i]
    for j in range(loan.CurrentInstallment[i]):
        unpaid_principal = unpaid_principal - monthly_lo[i] + unpaid_principal*loan.YearlyInterestRate[i]/12
    EAD_loan[i] = unpaid_principal
    

In [166]:
#total provision
# total provision for motgage
provision_mort = 0
for i in range(len_mortgage):
    if mortgage.PD[i] < 0.003:
        mortgage.PD[i] = 0.003
    if mortgage.LGD[i] < 0.1:
        mortgage.LGD[i] = 0.1
    provision_mort += EAD_mort[i] * mortgage.PD[i] * mortgage.LGD[i]
 

# total provision for loan
provision_loan = 0
for i in range(len_loan):
    if loan.PD[i] < 0.003:
        loan.PD[i] = 0.003
    provision_loan += EAD_loan[i] * loan.PD[i]


# total provsion for bond
provision_bond = 0
for i in range(len_bond):
    l = mydata.PresentPrice[i]*mydata.BondsHeld[i] * 0.45 * mydata.PD[i]
    if not(math.isnan(l)):
        provision_bond += l
    
    
print(f"The total provision for bonds is:{provision_bond}")
print(f"The total provision for mortgage is:{provision_mort}")
print(f"The total provision for loan is: {provision_mort}")
      
    
    
    

    
    
    

C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Yitia\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


The total provision for bonds is:3285486948.220779
The total provision for mortgage is:51993767436.4706
The total provision for loan is: 51993767436.4706


In [158]:
# part c
# bond
RWA_bond = 0
for i in range(len_bond):
    n = bond.PD[i]
    P = mydata.PresentPrice[i] * mydata.BondsHeld[i]
    if not(math.isnan(P)):
        if (n < 0.01):
            RWA_bond += 0.2 * P
        elif (n >= 0.01 and n < 0.12):
            RWA_bond += 0.5 * P
        elif (n >= 0.12 and n < 0.36):
            RWA_bond += 1 * P
        else:
            RWA_bond += 1.5 * P



In [159]:
RWA_bond

24267838427.75117

In [160]:
#mortgage
RWA_mortgage = 0
for i in range(len_mortgage):
    RWA_mortgage += 0.35 * EAD_mort[i]
    
    
RWA_mortgage

138231801567.4811

In [161]:
# loan
RWA_loan = 0
for i in range(len_loan):
    RWA_loan += 0.75 * EAD_loan[i]
    
RWA_loan

211059462.14277735

In [162]:
RWA = RWA_bond + RWA_mortgage + RWA_loan
print(f"the total risk-weighted assets of the bank is:{RWA}")

the total risk-weighted assets of the bank is:162710699457.37506


In [163]:
cap_ad_requiremnet = RWA * 0.115
print(f"the risk adequacy capital of this bank is:{cap_ad_requiremnet}")

the risk adequacy capital of this bank is:18711730437.598133
